In [1]:
import nfl_data_py as nfl


In [3]:
import nfl_data_py as nfl
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)

years = list(range(2010, 2024))

try:
    # Fetch weekly rosters data
    data = nfl.import_weekly_rosters(years)
    
    # Check for duplicate indices in the data
    if data.index.duplicated().any():
        logging.error("Duplicate indices found in the rosters DataFrame.")

    # Log the shape and index details
    logging.debug(f"Data shape: {data.shape}")
    logging.debug(f"Data index: {data.index}")
    
    # Check if 'birth_date' column exists before processing
    if 'birth_date' in data.columns:
        # Attempt to calculate the 'age' column
        try:
            roster_dates = data['gameday']  # Assuming 'gameday' is a column in the data
            data['age'] = ((pandas.to_datetime(roster_dates) - pandas.to_datetime(data['birth_date'])).dt.days / 365.25).round(3)
            logging.debug("Age column successfully calculated.")
        except Exception as e:
            logging.error(f"Failed to calculate the 'age' column: {e}")
    else:
        logging.warning("'birth_date' column not found in the data.")
    
except ValueError as ve:
    logging.error(f"ValueError occurred: {ve}")
except Exception as e:
    logging.error(f"An unexpected error occurred: {e}")

ERROR:root:ValueError occurred: cannot reindex on an axis with duplicate labels
